# Decision trees

build one!

In [5]:
import os

In [16]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()

X = iris.data
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=3)

tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [17]:
from sklearn.tree import export_graphviz

export_graphviz(tree_clf,
                out_file=f"{os.getcwd()}/iris_tree.dot",
                feature_names=iris.feature_names,
                class_names=iris.target_names,
                rounded=True,
                filled=True
               )

In [19]:
%%bash

dot -Tpng iris_tree.dot -o iris_tree.png

![tree](iris_tree.png "decision tree")

Decision Trees do not take much in the way of data prep and are intutative, these are the main advantages.

## Gini impurity

gini ($G_i$) gives the impurity of the $ith$ node. So $G = 0$ is best. 

$$G_i = 1 - \sum^{n}_{k=1}p_{i,k}^2$$

$p_{i,k}$ is the ratio of class k instances among training instances in the $i^{th}$ node

In [22]:
class_1 = class_2 = [x for x in range(51)]

In [24]:
import itertools as it

In [25]:
def calc_gini(classes, max_classes):
    
    ps = [(k/k_max)**2 
          for k, k_max in zip(classes, max_classes)
         ]
    return 1- sum(ps)
        

In [ ]:
for k_1, k_2 in it.product(class_1, class_2):
    